In [1]:
import pandas as pd
import time
from tqdm import tqdm
import numpy as np

In [2]:
base_url = 'https://dom.mingkh.ru'

In [3]:
for page in tqdm(range(1,328)):
    time.sleep(np.random.poisson(5))
    page_table = pd.read_html(f'https://dom.mingkh.ru/moskva/moskva/houses?page={page}',extract_links='body')[0]
    page_table[['Адрес','home_url']] = page_table['Адрес'].apply('|'.join).str.split('|',expand=True)
    page_table.assign(page=page).to_csv('base_home_data.csv',mode='a')

In [4]:
dd = pd.read_csv('base_home_data.csv')

In [5]:
def parse_home_page(home_url,id):
    df_list = pd.read_html(home_url)
    home_data = pd.concat(df_list,axis=0)\
                    .iloc[:,:3].dropna(thresh=1)\
                        .drop_duplicates(subset=0)\
                            .set_index(0).fillna('')\
                                .astype(str).apply(''.join,axis=1)\
                                    .rename(id)
    return home_data.rename_axis('id').to_frame().T

In [16]:
import httpx
import asyncio
from io import BytesIO
from tqdm import tqdm

In [8]:
dd.shape

(32933, 9)

In [9]:
home_list = dd['home_url'].where(lambda x: x!='home_url').dropna().drop_duplicates()

In [30]:
result_list = []
for idx,url in tqdm(enumerate(home_list,1),total=len(home_list)):
    result_list.append(parse_home_page(base_url+url,url.split('/')[-1]))
    if idx % 500 ==0:
        pd.concat(result_list).to_hdf('detailed_data.h5',key=str(idx))
        result_list = []

  2%|▏         | 499/32551 [02:36<3:01:23,  2.95it/s]/home/charubaiel/.local/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '500'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/tmp/ipykernel_810/2103563321.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Год ввода в эксплуатацию', 'Детская площадка', 'Дом признан аварийным',
       'Количество жилых помещений', 'Класс энергетической эффективности',
       'Количество лифтов', 'Количество подъездов',
       'Наибольшее количество этажей', 'Наименьшее количество этажей',
       'Количество нежилых помещений', 'Площадь жилых помещений м2',
       'Площадь нежилых помещений м2', 'Площа

In [14]:
# async def get_url(session,url):
#     full_url = base_url + url       
#     response = await session.get(full_url)

#     if response.status_code == 200:
#         return response
#     print(response.status_code)

# async with httpx.AsyncClient(http2=True,timeout=30) as client:
#     tasks = []
#     for url in home_list.drop_duplicates().values:
#         tasks.append(get_url(client,url))

#     response_list = await asyncio.gather(*tasks,return_exceptions=True)